# Manipulating models

In [36]:
from cameo import load_model
model = load_model('data/e_coli_core.xml.gz')

## Traveling in time

Usually one relies on making copies if objects need to be changed but the original state needs to be retained. Unfortunately, making copies of models is time consuming.

In [37]:
model.solve()

,fluxes,reduced_costs
ACALD,0.000000,8.673617e-19
ACALDt,0.000000,0.000000e+00
ACKr,0.000000,-1.734723e-18
ACONTa,6.007250,0.000000e+00
...,...,...
THD2,0.000000,-2.546243e-03
TKT1,1.496984,0.000000e+00
TKT2,1.181498,0.000000e+00
TPI,7.477382,0.000000e+00


In [38]:
%%time
copy_of_model = model.copy()

CPU times: user 32.4 ms, sys: 7.85 ms, total: 40.2 ms
Wall time: 49.8 ms


Yes, even milliseconds add up pretty quickly if you need to run many simulation (e.g. if you need to knock out every single gene individually in the model to check if it is essential or not).

In [39]:
%%time
for gene in model.genes:
    mutant = model.copy()
    mutant.genes.get_by_id(gene.id).knock_out()

CPU times: user 7.15 s, sys: 575 ms, total: 7.72 s
Wall time: 12.1 s


For that reason cameo provides a mechanism that is less time consuming. Some methods (e.g. `knock_out`) provide a keyword argument that allows the user to provide a `TimeMachine` object (`tm`). TimeMachine can be used in combination with Python's `with` statement in the same way you would use it open a file for example.

In [40]:
from cameo.util import TimeMachine

In [41]:
%%time
with TimeMachine() as tm:
    for gene in model.genes:
        gene.knock_out(time_machine=tm)

CPU times: user 42.5 ms, sys: 1.32 ms, total: 43.8 ms
Wall time: 47 ms


This is very similar to opening, reading and closing of files in Python.

In [42]:
with open('data/ecoli.csv', 'r') as file:
    print(file.read()[0:103])

bnumber,genes
b0001,thrL
b0002,thrA
b0002,thrA1
b0002,thrA2
b0003,thrB
b0004,thrC
b0005,yaaX
b0006,yaaA


Once one leaves the scope of `with` (the lines start again one indentation level to the right), everything is rolled back (the open file is closed; the gene knockout is undone).

## Changing flux bounds

One can change the flux bounds of a reaction using the 'change_bounds' methods. For example let's set the lower and upper bound of phosphoglycerate kinase to 0 (effictively knocking out the reaction).

In [43]:
with TimeMachine() as tm:
    model.reactions.PGK.change_bounds(lb=0, ub=0, time_machine=tm)
    print("PGK's bounds inside the with statement")
    print(model.reactions.PGK.lower_bound, model.reactions.PGK.upper_bound)
    print('Mutant growth rate: ', model.solve().objective_value)
print("PGK's bounds outside the with statement")
print(model.reactions.PGK.lower_bound, model.reactions.PGK.upper_bound)

PGK's bounds inside the with statement
0 0
Mutant growth rate:  0.0
PGK's bounds outside the with statement
-1000.0 1000.0


## Changing the medium

One can access the medium condition using `model.medium`. Lower and upper bounds of exchanges reactions are used to set a certain medium condition.

In [44]:
model.medium

,reaction_id,reaction_name,lower_bound,upper_bound
0,EX_co2_e,CO2 exchange,-1000.0,1000.0
1,EX_glc__D_e,D-Glucose exchange,-10.0,1000.0
2,EX_h_e,H+ exchange,-1000.0,1000.0
3,EX_h2o_e,H2O exchange,-1000.0,1000.0
4,EX_nh4_e,Ammonia exchange,-1000.0,1000.0
5,EX_o2_e,O2 exchange,-1000.0,1000.0
6,EX_pi_e,Phosphate exchange,-1000.0,1000.0


Changing the carbon source in the medium can be achieved by adjusting the flux bounds of the respective exchange reactions appropriately. For example, the following code block removes glucose from the medium and adds succinate.

In [45]:
with TimeMachine() as tm:
    model.reactions.EX_glc__D_e.change_bounds(lb=0, ub=0, time_machine=tm)
    model.reactions.EX_succ_e.change_bounds(lb=-10, ub=0, time_machine=tm)
    solution = model.solve()
    print(model.reactions.BIOMASS_Ecoli_core_w_GAM.flux)

0.39756301542776273


Changing the carbon source to succinate led to a significant drop in growth rate.

## Exercise

* Change the carbon source in the medium to a different carbon source. What is the difference in the growth rate observed?
* How about growing E. coli under anaerobic conditions?

## Adding reactions and pathways

In [46]:
from cameo import Reaction, Metabolite

Ok, let's create a new reactions.

In [47]:
new_reaction = Reaction('alchemy')

This reaction is going to convert water into gold. So we need to create a new metabolite, since gold is not yet part of _E. coli's_ native metabolism.

In [27]:
gold = Metabolite(id='gold_c', compartment='c')

Now, we're going to specify the reaction's stoichiometry.

In [28]:
new_reaction.add_metabolites({model.metabolites.h2o_c: -1, gold: 1})

Printing the reaction reveals that the reaction indeed converts water into gold.

In [29]:
print(new_reaction.build_reaction_string())

h2o_c --> gold_c


Now, let's add the new reaction to the model.

In [30]:
model.add_reaction(new_reaction)

Quickly check that the reaction was indeed added to the model.

In [32]:
model.reactions.alchemy

Id,alchemy
Name,
Stoichiometry,h2o_c --> gold_c
GPR,
Lower bound,0.000000
Upper bound,1000.000000


Let's produce some gold then!

In [16]:
model.objective = model.reactions.alchemy
model.solve().objective_value

0.0

:-(

What happened? Forgot to add an exchange reaction so that gold can leave the system.

In [33]:
model.add_demand(model.metabolites.gold_c)

Id,DM_gold_c
Name,Demand
Stoichiometry,gold_c -->
GPR,
Lower bound,0.000000
Upper bound,1000.000000


Yes, much better!

In [34]:
model.objective = model.reactions.alchemy
model.solve().objective_value

1000.0

## Exercise

* Add a pathway to the model (ideally one that you're personally interested in; you can also use a different model if you like). 